In [1]:
# This is its own cell because it takes a while to load this thing
from gensim import models

# takes a little bit. increase limit at own risk.
# model = models.KeyedVectors.load_word2vec_format('./GoogleNews-vectors-negative300.bin', binary=True, limit=500000)  
model = models.KeyedVectors.load_word2vec_format('./GoogleNews-vectors-negative300.bin', binary=True, limit=500000)  

In [7]:
# imports and list of scrape times.

import sklearn
import requests
import json
import _pickle as pickle
import os
import numpy as np
import time
import shutil
import datetime
import cv2
import urllib.request
import sqlite3 as sql
import base64
import ffmpy3
from file_read_backwards import FileReadBackwards

list_of_scrape_times = []

In [8]:
## class Loader

class Loader:
    def __init__(self, list_of_ids = [], list_of_times = []):
        self.list_of_ids = list_of_ids
        self.list_of_times = list_of_times
    
    filedir = ""

    ## 'file' omitted because it's not used in generating features
    list_of_types = ['text','log','contact','calender','gps','tweets','Instagram', 'Instagram media', 'audio','phq']
    
    def deletdatboi(self, nombre):
        shutil.rmtree('./datafor' + list_of_scrape_times[nombre])
        
    def lids(self):
        # connect to file
        conn = sql.connect('phonedata.db')
        # create cursor for making calls to database
        c = conn.cursor()

        list_of_approved_ids = []
        text_file = open("codes.txt", "r")
        list_of_approved_ids = text_file.read().split('\n')
        del list_of_approved_ids[-1]
        
        self.list_of_ids = []
        
        for row in c.execute('SELECT * FROM ids'):
            # one result
            if (row[0] in list_of_approved_ids):
                self.list_of_ids.append(row[0])
        
        return self.list_of_ids
    
    def lits(self):
        # connect to file
        conn = sql.connect('phonedata.db')
        # create cursor for making calls to database
        c = conn.cursor()

        self.list_of_times = []
        
        for row in c.execute('SELECT * FROM ids'):
            # one result
            self.list_of_times.append(row[1])
        return self.list_of_times
    
    def downloadAndLabel(self):
        # connect to file
        conn = sql.connect('phonedata.db')
        # create cursor for making calls to database
        c = conn.cursor()

        self.list_of_ids = []
        
        for row in c.execute('SELECT * FROM ids'):
            # one result
            self.list_of_ids.append(row[0])
            self.list_of_times.append(row[1])

        number_cols = len(self.list_of_types)
        number_rows = len(self.list_of_ids)

        ### create directory for this particular scrape/pull
        timenow  = str(int(time.time())) # for temporal congruency
        timereadable = datetime.datetime.fromtimestamp(int(timenow)).strftime('%H:%M')
        list_of_scrape_times.append(timereadable)
        os.mkdir('./datafor' + timereadable)
        Loader.filedir = './datafor' + timereadable

        # alex gave me a list of approved mturk ids
        list_of_approved_ids = []
        text_file = open("codes.txt", "r")
        list_of_approved_ids = text_file.read().split('\n')
        del list_of_approved_ids[-1]
        
        for i in range(0,number_rows):
            for j in range(0,number_cols):
                
                if (self.list_of_ids[i] not in list_of_approved_ids):
                    break
                
                apickle = []

                exampleLookup = (self.list_of_ids[i], self.list_of_types[j])
                for row in c.execute('SELECT DISTINCT* FROM data WHERE id=? AND type=?', exampleLookup):
                    # the row with the ID, type, and content
                    apickle.append(row[2])
                    
                    # WEIRD: duplicate ids in data table with different data
                    if exampleLookup[1] == "phq" or exampleLookup[1] == "audio":
                        break

                # print(apickle)    
                
                pickle.dump(apickle, open( Loader.filedir  + "/DP" + self.list_of_ids[i] +  self.list_of_types[j] + ".p", "wb" ))
        
        ## OLD CODE FOR INTERFACING THROUGH THE WEBSERVER
        ## CODE ABOVE ACCESSES A .DB FILE LOCALLY
        '''
        
        r = requests.get('http://depressionmqp.wpi.edu:8080/getids')
        list_of_idtime = r.json()
        

        for i in range(0,len(list_of_idtime)):
            self.list_of_ids.append( list_of_idtime[i]['id'].encode('ascii','ignore') )

        for i in range(0,len(list_of_idtime)):
            self.list_of_times.append( list_of_idtime[i]['date'] )

        number_cols = len(self.list_of_types)
        number_rows = len(self.list_of_ids)
        
        ### create directory for this particular scrape/pull
        timenow  = str(int(time.time())) # for temporal congruency
        timereadable = datetime.datetime.fromtimestamp(int(timenow)).strftime('%H:%M')
        list_of_scrape_times.append(timereadable)
        os.mkdir('./datafor' + timereadable)
        Loader.filedir = './datafor' + timereadable


        for i in range(0,number_rows):
            for j in range(0,number_cols):
                
                temp = requests.get('http://depressionmqp.wpi.edu:8080/getdata?id=' + str(int(self.list_of_ids[i])) + '&type=' + self.list_of_types[j])
                fintemp = json.loads(temp.text)["data"]
                
                while(json.loads(temp.text)["nextURL"] != ''):
                    temp = requests.get('http://depressionmqp.wpi.edu:8080' + json.loads(temp.text)["nextURL"])
                    fintemp += json.loads(temp.text)["data"]
                
                pickle.dump(fintemp, open( Loader.filedir + "/DP" + str(int(self.list_of_ids[i])) +  self.list_of_types[j] + ".p", "wb" ))

                # loads it into memory. i will not use this for now
                # for sake of architectural sanity
                # list_of_jsons[i][j] = temp
        '''

In [9]:
## Featurizer featurizes methods that convert json objects of the appropriate type into features

class Featurizer:
    def __init__(self):
        pass
        #self.name = name

    # takes in text pickle and scrapedate, returns vector of 14 elements
    # the first element is count of texts sent in the 24 hours before the scrape
    # the last element is count of texts sent on the 24 hour window 14 days before the scrapedate
    def textFreqVec14(self, text, scrapedate):
        
        textFreqVec = np.zeros((14,))
        
#         if(len(text)==0):
#             textFreqVec[:] = np.nan
#             return textFreqVec
        
        mseconds_in_twoweeks = 1209600000;
        mseconds_in_day = 86400000;
        
        # assuming unordered texts (WHICH TURNS OUT IS THE CASE)
        for day in range(0,14):
            time_during_week_ub = scrapedate - ((day)*mseconds_in_day)    
            time_during_week_lb = scrapedate - ((day+1)*mseconds_in_day)
            for t in range(0,len(text)):
                text_date = int(json.loads(text[t])['date'].encode('ascii','ignore'))
                if ((time_during_week_ub > text_date) and (text_date > time_during_week_lb)):
                    textFreqVec[day] += 1.0
                    
        return textFreqVec
    

    # takes in call pickle and scrapedate, returns vector of 14 elements
    # the first element is count of calls sent in the 24 hours before the scrape
    # the last element is count of calls sent on the 24 hour window 14 days before the scrapedate
    def callFreqVec14(self, call, scrapedate):
        
        callFreqVec = np.zeros((14,))
        
#         if(len(call)==0):
#             callFreqVec[:] = np.nan
#             return callFreqVec
        
        mseconds_in_twoweeks = 1209600000;
        mseconds_in_day = 86400000;
        
        for day in range(0,14):
            time_during_week_ub = scrapedate - ((day)*mseconds_in_day)    
            time_during_week_lb = scrapedate - ((day+1)*mseconds_in_day)
            for c in range(0,len(call)):
                call_date = int(json.loads(call[c])['date'].encode('ascii','ignore'))
                if ((time_during_week_ub > call_date) and (call_date > time_during_week_lb)):
                    callFreqVec[day] += 1.0
                    
        return callFreqVec
    
    
    # input is tweets pickle, return master vector
    def embeddingToMastersum(self, tweets):
        
        masterSum = np.zeros((300,))

        # add every word vector into master sum
        for i in range(0,len(tweets)):
            try:
                masterSum += self.tweetToEmbedding(tweets[i])
            except:
                pass
        
        return masterSum

    
    # input is one single tweet, returns vector embedding of entire tweet.
    # eg: responseobject.json()[0]
    def tweetToEmbedding(self, tweet):

        q = tweet['text'].split()
        
        sumVector = np.zeros((300,))
        
        # turn every word into embedding for 1 tweet, add all vectors
        for i in range(0,len(q)):
            try:
                sumVector += model[q[i]]
            except KeyError:
                pass
            except IndexError:
                pass
        
        return sumVector
    
    
    # input is tweets pickle, returns follow count
    def followerCount(self, tweets):

        followerCount = 0

#         if(len(tweets)==0):
#             followerCount = np.nan
#             return followerCount
        
        # get
        try:
            followerCount = json.loads(tweets[0])['user']['followers_count']
        except KeyError:
            pass
        except IndexError:
            pass

        return followerCount

    # input is tweets pickle, returns friend count
    def followingCount(self, tweets):

        followingCount = 0

#         if(len(tweets)==0):
#             followingCount = np.nan
#             return followingCount
        
        try:
            followingCount = json.loads(tweets[0])['user']['friends_count']
        except KeyError:
            pass
        except IndexError:
            pass

        return followingCount

    # input is tweets pickle, return avg likes per post for the last 2 weeks
    def twitterLikeFreq(self, tweets, scrapedate):

        twitLikeVec = np.zeros((1,))
        
#         if(len(tweets)==0):
#             twitLikeVec[:] = np.nan
#             return twitLikeVec
        
        mseconds_in_twoweeks = 1209600000;
        mseconds_in_day = 86400000;
        
        # upper and lower bounds
        time_during_week_ub = scrapedate - ((0)*mseconds_in_day)    
        time_during_week_lb = scrapedate - ((14)*mseconds_in_day)
        for t in range(0,len(tweets)):

            utc = json.loads(tweets[t])['created_at']

            tweet_date = int(time.mktime(time.strptime(utc,"%a %b %d %H:%M:%S +0000 %Y"))) * 1000

            if ((time_during_week_ub > tweet_date) and (tweet_date > time_during_week_lb)):
                twitLikeVec[0] += 1.0
                    
        if(twitLikeVec[0] == 0):
            return np.zeros((1,))
                    
        return twitLikeVec/14


    # input is tweets pickle, return avg retweets per post for the last 2 weeks
    def twitterRetweetFreq(self, tweets, scrapedate):

        twitRTVec = np.zeros((1,))
        
#         if(len(tweets)==0):
#             twitRTVec[:] = np.nan
#             return twitRTVec
        
        mseconds_in_twoweeks = 1209600000;
        mseconds_in_day = 86400000;
        
        # upper and lower bounds
        time_during_week_ub = scrapedate - ((0)*mseconds_in_day)    
        time_during_week_lb = scrapedate - ((14)*mseconds_in_day)
        for t in range(0,len(tweets)):

            utc = json.loads(tweets[t])['created_at']

            tweet_date = int(time.mktime(time.strptime(utc,"%a %b %d %H:%M:%S +0000 %Y"))) * 1000

            if ((time_during_week_ub > tweet_date) and (tweet_date > time_during_week_lb)):
                twitRTVec[0] += json.loads(tweets[t])['favorite_count']
                    
        return twitRTVec/14
    
    # input is contacts pickle, returns number of contacts
    def numOfContacts(self, contacts):
        
        return len(contacts)
    
    # input is instagram pickle, return two features: follows count, followed by count  
    def instagramThings(self, instagram):
        
        followsFollowed = np.zeros((2,))
        
        try:
            if(type(json.loads(instagram[0])) == str):
                print("DAMN YOU DAMON!!(ig)")
            else: # its a dictionary like it's supposed to be
                followsFollowed[0] = json.loads(instagram[0])['data']['counts']['follows']
                followsFollowed[1] = json.loads(instagram[0])['data']['counts']['followed_by']
        except IndexError:
            pass
        
        return followsFollowed
    
    # takes in instagramMedia pickle scrape date, spits out filter usage frequency for the past 2 weeks
    def instagramFilterFreq(self, instagramMedia, scrapedate):
        
        instaFiltVec = np.zeros((1,))
        
        mseconds_in_twoweeks = 1209600000;
        mseconds_in_day = 86400000;
        
        # upper and lower bounds
        time_during_week_ub = scrapedate - ((0)*mseconds_in_day)    
        time_during_week_lb = scrapedate - ((14)*mseconds_in_day)
        for t in range(0,len(instagramMedia)):

            if(instagramMedia != '[object Object]'):
                igpost_date = int(json.loads(instagramMedia[t])['created_time']) * 1000
            else:
                print("DAMN YOU DAMON!!(ig)")

            if ((time_during_week_ub > igpost_date) and (igpost_date > time_during_week_lb)):
                instaFiltVec[0] += 1.0
                    
       
        nofilter_count = 0

        for i in range(0,int(instaFiltVec[0])):
            if(json.loads(instagramMedia[i])['filter'] == "Normal"):
                nofilter_count += 1
        
        instaFiltVec[0] -= nofilter_count
        
        return instaFiltVec
        
       
    
    # takes in instagramMedia pickle and scrape date, returns a vector that
    # contains a normalized percentage (0-1) for the usage of the filters
    # listed below for the past 2 weeks: 
    # 'Amaro': 0,
    #  'Crema': 0,
    #  'Hefe': 5,
    #  'Inkwell': 0,
    #  'Rise': 0,
    #  'Valencia': 0,
    #  'Willow': 0,
    #  'X-Pro II': 0}
    
    # output example: [0.5,0,0,0,0,0.5,0,0] 
    # interpretation: user used valencia half the time, Willow the other half
    # of time, for the past 2 weeks of Instagram posts.1`sas
    def instagramFilterVector(self, instagramMedia, scrapedate):
        
        filters = {'Valencia':0,'X-Pro II':0, 'Hefe':0, 'Amaro':0, 'Rise':0, 'Willow':0, 'Crema':0, 'Inkwell':0}
        filtervec = np.zeros((8,))
        
        numposts = 0
        
        mseconds_in_twoweeks = 1209600000;
        mseconds_in_day = 86400000;
        
        # upper and lower bounds
        time_during_week_ub = scrapedate - ((0)*mseconds_in_day)    
        time_during_week_lb = scrapedate - ((14)*mseconds_in_day)
        for t in range(0,len(instagramMedia)):

            utc = json.loads(instagramMedia[t])['created_time']
            
            if(instagramMedia != '[object Object]'):
                igpost_date = int(json.loads(instagramMedia[t])['created_time']) * 1000
            else:
                print("DAMN YOU DAMON!!(ig)")

            if ((time_during_week_ub > igpost_date) and (igpost_date > time_during_week_lb)):
                numposts += 1
                
        if(numposts == 0):
            return np.zeros((8,))
                
        for i in range(0,numposts):
            filt = json.loads(instagramMedia[i])['filter']
            if(filt in filters):
                filters[filt] += 1
            else:
                pass
            
        for i in range(0,8):
            filtervec[i] = filters[list(filters)[i]]

        # percentage of how much the filters are used as a normalized vector
        return filtervec/(numposts)
    
    # takes in InstagramMedia, returns comment and like frequency for the 
    # past 2 weeks.
    def instagramLikeComFreq(self, instagramMedia, scrapedate):
        
        counts = np.zeros((2,))
        
        postcount = 0
        
        mseconds_in_twoweeks = 1209600000;
        mseconds_in_day = 86400000;
        
        # upper and lower bounds
        time_during_week_ub = scrapedate - ((0)*mseconds_in_day)    
        time_during_week_lb = scrapedate - ((14)*mseconds_in_day)
        for t in range(0,len(instagramMedia)):

            utc = json.loads(instagramMedia[t])['created_time']
            
            if(instagramMedia != '[object Object]'):
                igpost_date = int(json.loads(instagramMedia[t])['created_time']) * 1000
            else:
                print("DAMN YOU DAMON!!(ig)")

            if ((time_during_week_ub > igpost_date) and (igpost_date > time_during_week_lb)):
                postcount += 1
        
        if(postcount == 0):
            return np.zeros((2,))        
        
        for i in range(0,postcount):
            counts[0] += json.loads(instagramMedia[i])['likes']['count']
            counts[1] += json.loads(instagramMedia[i])['comments']['count']
        
        if (counts[0] + counts[1] == 0):
            return np.zeros((2,))
        else:
            # likes per post for past 2 weeks
            return counts/postcount


    # takes in instagramMedia pickle and scrapedate, returns IG post 
    # frequency for the past 2 weeks
    def instagramPostFreq(self, instagramMedia, scrapedate):
        
        postcount = np.zeros((1,))
        
        mseconds_in_twoweeks = 1209600000;
        mseconds_in_day = 86400000;
        
        # upper and lower bounds
        time_during_week_ub = scrapedate - ((0)*mseconds_in_day)    
        time_during_week_lb = scrapedate - ((14)*mseconds_in_day)
        for t in range(0,len(instagramMedia)):

            utc = json.loads(instagramMedia[t])['created_time']
            
            if(instagramMedia != '[object Object]'):
                igpost_date = int(json.loads(instagramMedia[t])['created_time']) * 1000
            else:
                print("DAMN YOU DAMON!!(ig)")

            if ((time_during_week_ub > igpost_date) and (igpost_date > time_during_week_lb)):
                postcount[0] += 1.0
                
        return postcount/14
        
        
    # takes instagramMedia, returns pixelwise average values for [H,S,V]
    # (Hue, Saturation, Value) for all posts in the past 2 weeks, the 
    # count of faces as a frequency of faces per picture, for the past
    # 2 weeks as well.
    
    # testvariable:
    # empty string: "" if not testing
    # 
    def averageHSVF(self, instagramMedia, scrapedate):


        counts = np.zeros((2,))
        
        postcount = 0
        
        mseconds_in_twoweeks = 1209600000;
        mseconds_in_day = 86400000;
        
        # upper and lower bounds
        time_during_week_ub = scrapedate - ((0)*mseconds_in_day)    
        time_during_week_lb = scrapedate - ((14)*mseconds_in_day)
        for t in range(0,len(instagramMedia)):

            utc = json.loads(instagramMedia[t])['created_time']
            
            if(instagramMedia != '[object Object]'):
                igpost_date = int(json.loads(instagramMedia[t])['created_time']) * 1000
            else:
                print("DAMN YOU DAMON!!(ig)")

            if ((time_during_week_ub > igpost_date) and (igpost_date > time_during_week_lb)):
                postcount += 1


        

        if(os.path.exists("./ILLSTOPBLINKINGSOON")):
            shutil.rmtree('./ILLSTOPBLINKINGSOON')

        os.mkdir('./ILLSTOPBLINKINGSOON')

        avgs = np.zeros((4,))

        if(postcount == 0):
            return avgs

        for i in range(0,postcount):
            url = json.loads(instagramMedia[i])['images']['thumbnail']['url']
            urllib.request.urlretrieve(url, './ILLSTOPBLINKINGSOON/' + str(i) + '.jpg')

        # face_cascade here is a pre trained classifier for frontal faces 
        face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
            
        avgHue = 0
        avgSatur = 0
        avgVal = 0
        avgFaces = 0

        ## GODS OF PROGRAMMING, FORGIVE ME FOR THIS TRIPLE NEST

        for k in range(0, postcount):
            ## BGR and not RGB because imread reads in BGR
            img = cv2.imread('./ILLSTOPBLINKINGSOON/' + str(k) + '.jpg')
            
            hsv = cv2.cvtColor(img,cv2.COLOR_BGR2HSV)
            
            grayImage = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            faces = face_cascade.detectMultiScale(grayImage,  scaleFactor=1.1, minNeighbors=5, flags = cv2.CASCADE_SCALE_IMAGE)
            
            avgFaces += len(faces)
            
            for i in range(0,hsv.shape[0]):
                for j in range(0,hsv.shape[1]):
                    avgHue += hsv[i,j,0]
                    avgSatur += hsv[i,j,1]
                    avgVal += hsv[i,j,2]
                    
        

        sums = [avgHue,avgSatur,avgVal]

        ## 22500 = 150x150 = instagram photo thumbnail shape
        avgs = list(map(lambda x: x/(22500*postcount), sums)).append(avgFaces/postcount)

        shutil.rmtree('./ILLSTOPBLINKINGSOON')

        return avgs      

    
    #input is texts pickle, return master vector 
    def embeddingToMastersumText(self, texts):
        
        masterSum = np.zeros((300,))

        # add every word vector into master sum
        for i in range(0,len(texts)):
            try:
                masterSum += self.textToEmbedding(texts[i])
            except:
                pass
        
        return masterSum

    
    # input is single text, returns vector embedding of entire text.
    def textToEmbedding(self, text):
        
        q = json.loads(text)["body"].split()

        sumVector = np.zeros((300,))

        # turn every word into embedding for 1 tweet, add all vectors
        for i in range(0,len(q)):
            try:
                sumVector += model[q[i]]
            except KeyError:
                pass
            except IndexError:
                pass

        return sumVector
    
    
    # returns [q1,q2,q3,q4,q5,q6,q7,q8,q9] and sum of all these scores
    def labelGenerator(self, phq):
        
        labelVector = np.zeros((10,))
        sumOfScores = 0
        
        # print(phq)
        
        for i in range(0,9):
            temp = int(json.loads(phq[0])['Q' + str(i)])
            labelVector[i] = temp
            labelVector[9] += temp
            
        return labelVector

        
    def voiceFeaturizer(self, voice):

        audiofeaturevec = np.zeros((1583,))

        if(len(voice) == 0):
            return audiofeaturevec

        # base64 string -> bitstring -> bitstream -> write into 3gp file
        bytestream = base64.b64decode(voice[0])
        fh = open("audio.3gp","wb")
        fh.write(bytestream)
        fh.close()

        # wav -> 3gp
        ff = ffmpy3.FFmpeg( inputs={'audio.3gp': None}, outputs={'audio.wav': None})
        ff.run()

        os.remove("audio.3gp")

        # call to openSMILE
        os.system( os.getcwd() + '/openSMILE-2.1.0/bin/linux_x64_standalone_static/SMILExtract -C ' + os.getcwd() + '/openSMILE-2.1.0/config/emobase2010.conf -I audio.wav -O "out.csv"')

        os.remove("audio.wav")

        # csv file has a giant header. last line contains the features we want
        # so we read the last line, cut out more useless string with 'l[9:]
        with FileReadBackwards("out.csv", encoding="utf-8") as frb:
            for l in frb:
                b = l[9:].split(',')
                break

        os.remove("out.csv")

        # make list of string into list of floats
        a = list(map(float, b))

        # make sklearn happy
        for i in range(0,len(a)):
            audiofeaturevec[i] = a[i]

        return audiofeaturevec

In [44]:
class Generator:

    def __init__(self, filedir, list_of_ids, list_of_times):
        self.filedir = filedir
        self.list_of_ids = list_of_ids
        self.f = Featurizer()
        self.num_of_people = len(list_of_ids)
        self.list_of_times = list_of_times
    
    BigMatrix = None
    audioless = None
    
    l = Loader()
    num_of_people = len(l.lids())
    
    # call frequency
    featureVectorCF = np.zeros((num_of_people,14))
    # text frequency
    featureVectorTF = np.zeros((num_of_people,14))
    # follower count
    featureVectorFC = np.zeros((num_of_people,1))
    # following count
    featureVectorFC2 = np.zeros((num_of_people,1))
    # twitter like frequency
    featureVectorTWL = np.zeros((num_of_people,1))
    # twitter retweet frequency
    featureVectorTWRT = np.zeros((num_of_people,1))
    # num of contacts
    featureVectorNC = np.zeros((num_of_people,1))
    # instagram follows, followed by
    featureVectorIG1 = np.zeros((num_of_people,2))
    # instagram filter usage freq
    featureVectorIG2 = np.zeros((num_of_people,1))
    # instagram filter vec: Valencia, X-Pro II, Hefe, Amaro, Rise, Willow, Crema, Inkwell
    featureVectorIGFV = np.zeros((num_of_people,8))
    # instagram like freq, comment freq
    featureVectorIGLC = np.zeros((num_of_people,2))
    # instagram post freq
    featureVectorIG3 = np.zeros((num_of_people,1))
    # instagram avg Hue, Saturation, Value, and total faces
    featureVectorHSVF = np.zeros((num_of_people,4))
    # audio features from openSMILE
    featureVectorAUD = np.zeros((num_of_people,1583))

    # labels, 0-8 for corresponding phq questions, last entry is sum of them.
    labelVector = np.zeros((num_of_people,10))
    
    # bag of words with word2vec (try last)
    #featureVectorTW = np.zeros((self.num_of_people,300))
    # bag of words with word2vec (try last)
    #featureVectorTW2V = np.zeros((self.num_of_people,300))
    

    def generateMatrix(self):

        # these are the feature vectors, one for every feature.
        # each feature vector contains the value for one feature
        # for every user. 

        
        # FILLING IN THE ABOVE VECTORS
        
        for i in range(0, self.num_of_people):
            
#             if(i%25 == 0):
#                 print(self.list_of_ids[i])
            
            ## The flow of operation is same under every title
            ## if data file is there and has actual data -> feature
            ## if data file is there and there is no data in it -> NaN
            ## if data file isnt there -> NaN
            
            ############# Call Log ###############################
            try:
                a1 = pickle.load( open( self.filedir + "/DP" + self.list_of_ids[i] +  "log" + ".p", "rb" ))
#                 if(len(a1)==0): 
#                     Generator.featureVectorCF[i:i+1,:] = np.nan
#                     continue
                        
                Generator.featureVectorCF[i] = self.f.callFreqVec14(a1, self.list_of_times[i])
                
            except FileNotFoundError:
                Generator.featureVectorCF[i:i+1,:] = np.nan
                pass
            
            
            ############# SMS Messages ###########################
            try:
                a2 = pickle.load( open( self.filedir + "/DP" + self.list_of_ids[i] +  "text" + ".p", "rb" )) 
#                 if(len(a2)==0): 
# #                     Generator.featureVectorTF[i:i+1,:] = np.nan
# #                     #featureVectorTW2V[i] = np.nan
# #                     continue
            
                Generator.featureVectorTF[i] = self.f.textFreqVec14(a2, self.list_of_times[i])
                #featureVectorTW2V[i] = self.f.embeddingToMastersumText(a2)
            
            except FileNotFoundError:
                Generator.featureVectorTF[i:i+1,:] = np.nan
                #featureVectorTW2V[i] = np.nan
                pass
            
            ############# Twitter ###############################
            try:
                a3 = pickle.load( open( self.filedir + "/DP" + self.list_of_ids[i] +  "tweets" + ".p", "rb" )) 
#                 if(len(a3)==0):
#                     #featureVectorTW[i] = np.nan
#                     Generator.featureVectorFC[i] = np.nan
#                     Generator.featureVectorFC2[i] = np.nan
#                     Generator.featureVectorTWL[i] = np.nan
#                     Generator.featureVectorTWRT[i] = np.nan
#                     continue
            
                #featureVectorTW[i] = self.f.embeddingToMastersum(a3)
                Generator.featureVectorFC[i] = self.f.followerCount(a3)
                Generator.featureVectorFC2[i] = self.f.followingCount(a3)
                Generator.featureVectorTWL[i] = self.f.twitterLikeFreq(a3, self.list_of_times[i])
                Generator.featureVectorTWRT[i] = self.f.twitterRetweetFreq(a3, self.list_of_times[i])
            
            except FileNotFoundError:
                #featureVectorTW[i] = np.nan
                Generator.featureVectorFC[i:i+1,:] = np.nan
                Generator.featureVectorFC2[i:i+1,:] = np.nan
                Generator.featureVectorTWL[i:i+1,:] = np.nan
                Generator.featureVectorTWRT[i:i+1,:] = np.nan
                pass
            
            ############# Contacts ###############################
            try:
                a4 = pickle.load( open( self.filedir + "/DP" + self.list_of_ids[i] +  "contact" + ".p", "rb" )) 
#                 if(len(a4)==0):
#                     Generator.featureVectorNC[i] = np.nan
#                     continue
                
                Generator.featureVectorNC[i] = self.f.numOfContacts(a4)
                
            except FileNotFoundError:
                Generator.featureVectorNC[i:i+1,:] = np.nan
                pass
            
            ############# Instagram ###############################
            try:
                a5 = pickle.load( open( self.filedir + "/DP" + self.list_of_ids[i] +  "Instagram" + ".p", "rb" )) 
#                 if(len(a5)==0): 
#                     Generator.featureVectorIG1[i] = np.nan
#                     continue
            
                Generator.featureVectorIG1[i] = self.f.instagramThings(a5)
                
            except FileNotFoundError:
                Generator.featureVectorIG1[i:i+1,:] = np.nan
                pass
            
            ############# Instagram Media #########################
            try:
                a6 = pickle.load( open( self.filedir + "/DP" + self.list_of_ids[i] +  "Instagram media" + ".p", "rb" ))
#                 if(len(a6)==0):
#                     Generator.featureVectorIG2[i] = np.nan
#                     Generator.featureVectorIGFV[i] = np.nan
#                     Generator.featureVectorIGLC[i] = np.nan
#                     Generator.featureVectorIG3[i] = np.nan
#                     Generator.featureVectorHSVF[i] = np.nan
#                     continue
                
                Generator.featureVectorIG2[i] = self.f.instagramFilterFreq(a6, self.list_of_times[i])
                Generator.featureVectorIGFV[i] = self.f.instagramFilterVector(a6, self.list_of_times[i])
                Generator.featureVectorIGLC[i] = self.f.instagramLikeComFreq(a6, self.list_of_times[i])
                Generator.featureVectorIG3[i] = self.f.instagramPostFreq(a6, self.list_of_times[i])
                Generator.featureVectorHSVF[i] = self.f.averageHSVF(a6, self.list_of_times[i])
            
            except FileNotFoundError:
                Generator.featureVectorIG2[i:i+1,:] = np.nan
                Generator.featureVectorIGFV[i:i+1,:] = np.nan
                Generator.featureVectorIGLC[i:i+1,:] = np.nan
                Generator.featureVectorIG3[i:i+1,:] = np.nan
                Generator.featureVectorHSVF[i:i+1,:] = np.nan
                pass
            
            ############# Audio #########################
#             try:
#                 a7 = pickle.load( open( self.filedir + "/DP" + self.list_of_ids[i] +  "audio" + ".p", "rb" ))
#                 if(len(a7)==0):
#                     Generator.featureVectorAUD [i] = np.nan
#                     continue
                
#                 Generator.featureVectorAUD [i] = self.f.voiceFeaturizer(a7)
            
#             except FileNotFoundError:
#                 Generator.featureVectorAUD [i:i+1,:] = np.nan
#                 pass
            
            ############# Label #########################
            try:
                b1 = pickle.load( open( self.filedir + "/DP" + self.list_of_ids[i] +  "phq" + ".p", "rb" )) 
#                 if(len(b1)==0):
#                     print("len = 0 " + self.list_of_ids[i])
#                     Generator.labelVector[i] = np.nan
#                     continue
            
                Generator.labelVector[i] = self.f.labelGenerator(b1)
            
            except FileNotFoundError:
                Generator.labelVector[i:i+1,:] = np.nan
                pass
            
                       
        Generator.BigMatrix = np.hstack((Generator.featureVectorCF, 
                                         Generator.featureVectorTF, 
                                         Generator.featureVectorFC, 
                                         Generator.featureVectorTWL, 
                                         Generator.featureVectorTWRT, 
                                         Generator.featureVectorNC, 
                                         Generator.featureVectorIG1, 
                                         Generator.featureVectorIG2, 
                                         Generator.featureVectorIGFV,
                                         Generator.featureVectorIGLC, 
                                         Generator.featureVectorIG3, 
                                         Generator.featureVectorHSVF, 
                                         Generator.featureVectorAUD, 
                                         Generator.labelVector))
        
        Generator.audioless = np.hstack((Generator.featureVectorCF, 
                                         Generator.featureVectorTF, 
                                         Generator.featureVectorFC, 
                                         Generator.featureVectorTWL, 
                                         Generator.featureVectorTWRT, 
                                         Generator.featureVectorNC, 
                                         Generator.featureVectorIG1, 
                                         Generator.featureVectorIG2, 
                                         Generator.featureVectorIGFV,
                                         Generator.featureVectorIGLC, 
                                         Generator.featureVectorIG3, 
                                         Generator.featureVectorHSVF, 
                                         Generator.labelVector))

#     CallF = None

#     CallF = featureVectorCF



In [45]:
# Everything I coded in one cell!

l = Loader()
# l.downloadAndLabel()

g = Generator('./datafor23:41', l.lids(), l.lits())
g.generateMatrix()
mtr = g.BigMatrix
g.BigMatrix.shape




(261, 1643)

In [46]:
mtr = g.audioless
mtr.shape

(261, 60)

In [47]:
## PREPROCESSING

from sklearn.preprocessing import Imputer

# replace missing values with mean of their corresponding features
# imp = Imputer(missing_values='NaN', strategy='mean', axis=0)
# mtr = imp.fit_transform(mtr)
# shuffle row-wise
np.random.shuffle(mtr)

data = mtr[:,0:14]
labels = mtr[:,50:61]

# normalize data (features now have gauss dist., 0 mean and unit variance)
data = sklearn.preprocessing.scale(data)


In [48]:
data[0:10,:]

array([[-0.45174364, -0.43120947, -0.46227116, -0.34143674, -0.37824797,
        -0.3667097 , -0.41995871, -0.27824165, -0.43403791, -0.34692746,
        -0.3699291 , -0.38043859, -0.36949047, -0.36860458],
       [-0.45174364, -0.43120947, -0.46227116, -0.34143674, -0.37824797,
        -0.3667097 , -0.41995871, -0.27824165, -0.43403791, -0.34692746,
        -0.3699291 , -0.38043859, -0.36949047, -0.36860458],
       [-0.45174364, -0.43120947, -0.46227116, -0.34143674, -0.37824797,
        -0.3667097 , -0.41995871, -0.27824165, -0.43403791, -0.34692746,
        -0.3699291 , -0.38043859, -0.36949047, -0.36860458],
       [-0.45174364, -0.43120947, -0.46227116, -0.34143674, -0.37824797,
        -0.3667097 , -0.41995871, -0.27824165, -0.43403791, -0.34692746,
        -0.3699291 , -0.38043859, -0.36949047, -0.36860458],
       [ 0.47768838, -0.30461141,  0.37910103, -0.34143674,  0.10021432,
        -0.3667097 ,  0.43413914, -0.20690465, -0.29260983, -0.09644871,
         0.20995976, -0.38

In [49]:
labels[0:10,:] # alex dis works and the top numbers are fukd

array([[ 2.,  1.,  3.,  2.,  3.,  1.,  1.,  0.,  0., 13.],
       [ 2.,  2.,  1.,  2.,  1.,  2.,  2.,  0.,  0., 12.],
       [ 1.,  1.,  3.,  1.,  0.,  1.,  0.,  0.,  0.,  7.],
       [ 2.,  3.,  2.,  3.,  3.,  3.,  3.,  2.,  1., 22.],
       [ 1.,  1.,  2.,  1.,  2.,  1.,  0.,  0.,  1.,  9.],
       [ 1.,  1.,  2.,  1.,  3.,  2.,  1.,  2.,  2., 15.],
       [ 3.,  3.,  3.,  3.,  3.,  3.,  1.,  2.,  3., 24.],
       [ 0.,  0.,  1.,  2.,  1.,  1.,  0.,  0.,  0.,  5.],
       [ 0.,  1.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  3.],
       [ 2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2., 18.]])

In [321]:
# DATA SPLIT (#nosnooping)

# TEST DATA (%15 percent of data)
numofppl_index = g.BigMatrix.shape[0] - 1
cut_index = int(g.BigMatrix.shape[0] * 0.85)

test_label = g.BigMatrix[cut_index:numofppl_index,:]
test_data = g.BigMatrix[cut_index:numofppl_index,:]

# TRAINING AND TEST DATA (%85 percent of data)

train_data = g.BigMatrix[0:cut_index,:]
train_label = g.BigMatrix[0:cut_index,:]


X = train_data[:,0:13]
y = train_label[:,1642:1643]#.reshape((376,))

mtrx = np.hstack((X,y))


mtrx = pd.DataFrame(mtrx)



mtrx = mtrx.dropna(axis=0, how='all')

In [323]:
y = train_label[:,1642:1643]#.reshape((376,))
y

array([[ 6.],
       [ 0.],
       [ 0.],
       [nan],
       [ 0.],
       [ 0.],
       [16.],
       [nan],
       [nan],
       [ 0.],
       [ 0.],
       [nan],
       [ 0.],
       [ 0.],
       [nan],
       [ 0.],
       [ 0.],
       [ 0.],
       [nan],
       [nan],
       [ 0.],
       [ 0.],
       [nan],
       [nan],
       [ 0.],
       [15.],
       [nan],
       [ 0.],
       [nan],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [nan],
       [ 0.],
       [ 0.],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [ 0.],
       [nan],
       [ 0.],
       [ 0.],
       [12.],
       [nan],
       [nan],
       [nan],
       [ 0.],
       [nan],
       [nan],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [nan],
       [ 0.],
       [nan],
       [nan],
       [nan],
       [ 0.],
       [nan],
       [ 0.],
       [ 0.],
       [ 0.],
      

In [320]:
mtrx[:,13:14]

array([[ 6.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [16.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [15.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [12.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [13.],
       [ 0.],
       [ 0.],
       [14.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
      

In [300]:
imp = Imputer(missing_values='NaN', strategy='mean', axis=0)
mtrx = imp.fit_transform(mtrx)

In [301]:
X = mtrx[:,0:13]
y = mtrx[:,13:14]

In [312]:
y.shape

(218,)

In [255]:
np.isnan(X).any()

0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
dtype: bool

In [215]:
np.isfinite(y).all()

0    False
dtype: bool

In [302]:

y = y.reshape(218,)
y.shape

(218,)

In [310]:
y/shape

(218,)

In [305]:
from sklearn.grid_search import GridSearchCV

c_range = list(range(1, 30))
parameters = {'kernel':('linear', 'rbf', 'poly', 'sigmoid'), 'C':c_range}
parameters['kernel']

from sklearn import svm

svc = svm.SVC()

grid = GridSearchCV(svc, parameters, cv=3, scoring='accuracy')
grid.fit(X, y)

/home/vape/.local/lib/python3.5/site-packages/sklearn/cross_validation.py:553: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of labels for any class cannot be less than n_folds=3.
  % (min_labels, self.n_folds)), Warning)


GridSearchCV(cv=3, error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'C': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29], 'kernel': ('linear', 'rbf', 'poly', 'sigmoid')},
       pre_dispatch='2*n_jobs', refit=True, scoring='accuracy', verbose=0)

In [306]:
grid.grid_scores_

print(grid.grid_scores_[0].parameters)
print(grid.grid_scores_[0].cv_validation_scores)
print(grid.grid_scores_[0].mean_validation_score)

{'kernel': 'linear', 'C': 1}
[0.88607595 0.94285714 0.91304348]
0.9128440366972477


In [99]:
from sklearn import svm, datasets
from sklearn.model_selection import GridSearchCV
iris = datasets.load_iris()
# parameters = {'kernel':('linear', 'rbf'), 'C':[1, 10]}
# svc = svm.SVC()
# clf = GridSearchCV(svc, parameters)
# clf.fit(iris.data, iris.target)

X = iris.data
y = iris.target

knn = KNeighborsClassifer

NameError: name 'KNeighborsClassifer' is not defined

In [93]:
iris.target

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [92]:
clf.grid_scores_

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [302]:
# save matrix as csv. 
import pandas as pd
# np.savetxt("foo.csv", g.featureMatrix , delimiter=",")

# another way

dff = pd.DataFrame(g.featureMatrix)
dff.to_csv("foo.csv")

In [303]:
z = pd.read_csv("foo.csv")
z.shape

(484, 2244)

In [203]:
for i in l.lids()[23:30]:
    b1 = pickle.load( open( "datafor15:20" + "/DP" + i +  "phq" + ".p", "rb" )) 
    print(b1)

['{"Q0":"3","Q1":"1","Q2":"0","Q3":"2","Q4":"0","Q5":"0","Q6":"0","Q7":"0","Q8":"0"}', '{"Q0":"1","Q1":"1","Q2":"0","Q3":"0","Q4":"0","Q5":"0","Q6":"0","Q7":"0","Q8":"0"}']
[]
['{"Q0":"0","Q1":"0","Q2":"0","Q3":"0","Q4":"0","Q5":"0","Q6":"0","Q7":"0","Q8":"0"}']
['{"Q0":"2","Q1":"1","Q2":"0","Q3":"0","Q4":"0","Q5":"0","Q6":"0","Q7":"0","Q8":"0"}']
['{"Q0":"3","Q1":"1","Q2":"3","Q3":"2","Q4":"1","Q5":"3","Q6":"3","Q7":"1","Q8":"0"}']
['{"Q0":"1","Q1":"1","Q2":"3","Q3":"3","Q4":"2","Q5":"2","Q6":"1","Q7":"2","Q8":"1"}']
['{"Q0":"2","Q1":"1","Q2":"1","Q3":"2","Q4":"0","Q5":"2","Q6":"2","Q7":"1","Q8":"2"}']


In [219]:
l.lids()[23:26]

['7276', '6830', '7664']

In [245]:
b1 = pickle.load( open( "datafor16:13" + "/DP" + "6578" +  "text" + ".p", "rb" )) 

print(json.loads(b1[0])["body"])

539833 is your Amazon security code.


In [235]:
apickle = []
exampleLookup = ("7664", "phq")
for row in c.execute('SELECT DISTINCT * FROM data WHERE id=? AND type=? ', exampleLookup):
            
    apickle.append(row[2])
    
    print(row)
    
apickle

('7664', 'phq', '{"Q0":"0","Q1":"0","Q2":"0","Q3":"0","Q4":"0","Q5":"0","Q6":"0","Q7":"0","Q8":"0"}')


['{"Q0":"0","Q1":"0","Q2":"0","Q3":"0","Q4":"0","Q5":"0","Q6":"0","Q7":"0","Q8":"0"}']

In [352]:
b1 = pickle.load( open( "datafor16:13" + "/DP" + "0660" +  "gps" + ".p", "rb" ))
b1

['<?xml version="1.0" encoding="UTF-8"?><kml xmlns="http://www.opengis.net/kml/2.2" xmlns:gx="http://www.google.com/kml/ext/2.2"><Document><name> Location history from 2018-01-04 to 2018-01-04 </name><open>1</open><description></description><StyleMap id="multiTrack"><Pair><key>normal</key><styleUrl>#multiTrack_n</styleUrl></Pair><Pair><key>highlight</key><styleUrl>#multiTrack_h</styleUrl></Pair></StyleMap><Style id="multiTrack_n"><IconStyle><Icon><href>https://earth.google.com/images/kml-icons/track-directional/track-0.png</href></Icon></IconStyle><LineStyle><color>99ffac59</color><width>6</width></LineStyle></Style><Style id="multiTrack_h"><IconStyle><scale>1.2</scale><Icon><href>https://earth.google.com/images/kml-icons/track-directional/track-0.png</href></Icon></IconStyle><LineStyle><color>99ffac59</color><width>8</width></LineStyle></Style></Document></kml>\n',
 '<?xml version="1.0" encoding="UTF-8"?><kml xmlns="http://www.opengis.net/kml/2.2" xmlns:gx="http://www.google.com/kml/e

In [24]:
## WASTELAND OF PLAY CELLS BELOW
import pickle

filedir = './datafor' + '288'

## access example

# instagram = pickle.load( open( filedir + "/DP" + str(int(l.lids()[4])) +  "tweets" + ".p", "rb" )) 

a3 = pickle.load( open( filedir + "/DP" + '1995377' +  "Instagram media" + ".p", "rb" )) 
# a3 = pickle.load( open( filedir + "/DP" + '19671950' +  "Instagram media" + ".p", "rb" )) 
#instagram = pickle.load( open( filedir + "/DP" + '11852603' +  "Instagram" + ".p", "rb" )) 



# json.loads(a3.json()[0])

a3.json()





[]

In [112]:
import requests
import json



# temp = requests.get('http://depressionmqp.wpi.edu:8080/getdata?id=' + str(98945548) + '&type=' + "text")
# fintemp = json.loads(temp.text)["data"]

# while(json.loads(temp.text)["nextURL"] != ''):
#     temp = requests.get('http://depressionmqp.wpi.edu:8080' + json.loads(temp.text)["nextURL"])
#     fintemp += json.loads(temp.text)["data"]

    
fintemp[8817]
    

'{"_id":"4","thread_id":"1","address":" 15085301734","person":"null","date":"1460240309687","date_sent":"0","protocol":"null","read":"1","status":"-1","type":"2","reply_path_present":"null","subject":"null","body":"Your face","service_center":"null","locked":"0","error_code":"0","seen":"1","deletable":"0","sim_slot":"0","sim_imsi":"null","hidden":"0","group_id":"null","group_type":"null","delivery_date":"null","app_id":"0","msg_id":"0","callback_number":"null","reserved":"0","pri":"0","teleservice_id":"0","link_url":"null","svc_cmd":"0","svc_cmd_content":"null","roam_pending":"0","spam_report":"0","safe_message":"0","sub_id":"-1","creator":"com.android.mms","secret_mode":"0","favorite":"0","d_rpt_cnt":"0","using_mode":"0","from_address":"null","announcements_subtype":"0","announcements_scenario_id":"null","device_name":"null"}'

In [525]:
kelsey = {'Valencia':0,'X-Pro II':0, 'Hefe':0, 'Amaro':0, 'Rise':0, 'Willow':0, 'Crema':0, 'Inkwell':0}

# for fil in kelsey:
#     print(kelsey[fil])

'Valencia' in kelsey

True

In [552]:
instagramMedia = pickle.load( open( filedir + "/DP" + '19671950' +  "Instagram media" + ".p", "rb" )) 
filters = {'Lark':0,'Slumber':0, 'Hefe':0, 'Amaro':0, 'Rise':0, 'Willow':0, 'Crema':0, 'Inkwell':0}
filtervec = np.ones((8,))

for i in range(0,3):
    filt = json.loads(a[i])['filter']
    if(filt in filters):
        filters[filt] += 1
    else:
        pass
    
for i in range(0,8):
    filtervec[i] = filters[list(filters)[i]]
    
filtervec

array([ 0.,  1.,  0.,  0.,  1.,  0.,  0.,  0.])

In [613]:
## TWITTER 70522438
## IG   19671950


# instagramMedia = pickle.load( open( filedir + "/DP" + '82562975' +  "Instagram media" + ".p", "rb" )) 
instagramMedia = pickle.load( open( filedir + "/DP" + '19671950' +  "Instagram media" + ".p", "rb" )) 

url = json.loads(instagramMedia.json()[0])['images']['thumbnail']['url']

from PIL import Image
import requests
from io import BytesIO

response = requests.get(url)
img = Image.open(BytesIO(response.content))


img.shape

AttributeError: 'JpegImageFile' object has no attribute 'shape'

In [520]:
filtervec = np.ones((8,))
filtervec/4

array([ 0.25,  0.25,  0.25,  0.25,  0.25,  0.25,  0.25,  0.25])

In [143]:
w = embeddingToMastersum(a3)
w.reshape((-1, 1)).T.shape


(1, 300)

In [156]:
timenow  = str(int(time.time())) # for temporal congruency
timenow1 = timenow
timenow1

'1511765588'

In [125]:
l.lits()

[1513024716885,
 1513026200334,
 1513026939825,
 1513027798716,
 1513028059396,
 1513028060408,
 1513029109178,
 1513034337824,
 1513034463419,
 1513035395273]

In [184]:

face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')


image = cv2.imread('./faces.jpg')
grayImage = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
#faces = face_cascade.detectMultiScale(grayImage)
faces = face_cascade.detectMultiScale(grayImage,  scaleFactor=1.1, minNeighbors=5, flags = cv2.CASCADE_SCALE_IMAGE)
print(len(faces))

#type(face_cascade) 

10


In [178]:
dir_file_list = os.listdir(".")
dir_file_list

['datafor19:07',
 'MLIntP3.ipynb',
 'ILLSTOPBLINKINGSOON',
 '.git',
 '.ipynb_checkpoints',
 'OpenCV Shenaniganry.ipynb',
 'README.md',
 'haarcascade_frontalface_default.xml',
 'faces.jpg',
 'glove.twitter.27B.zip',
 'MLInt.ipynb',
 'datafor690',
 'glove.twitter.27B.50d.txt',
 'GoogleNews-vectors-negative300.bin']

In [67]:
import sqlite3 as sql

# connect to file
conn = sql.connect('phonedata.db')
# create cursor for making calls to database
c = conn.cursor()

list_of_ids = []
list_of_times = []


for row in c.execute('SELECT * FROM ids'):
    # one result
    list_of_ids.append(row[0])
    list_of_times.append(row[1])
    # prints th id
#     print row[0]
#     # prints the timestamp
#     print row[1]


SyntaxError: EOL while scanning string literal (<ipython-input-374-44dcd50a8971>, line 1)

In [375]:
len(l)

TypeError: object of type 'Loader' has no len()

In [147]:
import pickle

a1 = pickle.load( open( "datafor15:06" + "/DP" + "0660" +  "tweets" + ".p", "rb" )) 

In [142]:
type(a1)

list

In [55]:
# l = Loader()
# list_of_ids = l.lids()
# for idd in range(0,len(list_of_ids)):
#     # print(phq)
#     phq = pickle.load(open( "datafor23:41" + "/DP" + str(list_of_ids[idd]) +  "phq" + ".p", "rb" ))
#     labelVector = np.zeros((10,))
#     sumOfScores = 0

#     # print(phq)

#     for i in range(0,9):
#         print(i)
#         temp = int(json.loads(phq[0])['Q' + str(i)])

#         labelVector[i] = temp
#         labelVector[9] += temp

#     #print(labelVector[9])


In [87]:
 try:
    exit(0)
catch